# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 12 2022 8:24AM,246110,10,NST-1115,"Amcyte Pharma, Inc.",Released
1,Sep 12 2022 8:21AM,246108,10,8515465,Beach Products Inc,Released
2,Sep 12 2022 8:21AM,246108,10,8515466,Beach Products Inc,Released
3,Sep 12 2022 8:21AM,246108,10,8515467,Beach Products Inc,Released
4,Sep 12 2022 8:21AM,246108,10,8515468,Beach Products Inc,Released
5,Sep 12 2022 8:21AM,246108,10,8515469,Beach Products Inc,Released
6,Sep 12 2022 8:21AM,246108,10,8515470,Beach Products Inc,Released
7,Sep 12 2022 8:21AM,246108,10,8515471,Beach Products Inc,Released
8,Sep 12 2022 8:21AM,246108,10,8515472,Beach Products Inc,Released
9,Sep 12 2022 8:21AM,246108,10,8515473,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
2,246102,Released,25
3,246103,Released,7
4,246104,Executing,1
5,246108,Released,13
6,246110,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246102,NaN,25.0
246103,NaN,7.0
246104,1.0,NaN
246108,NaN,13.0
246110,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246101,3.0,4.0
246102,0.0,25.0
246103,0.0,7.0
246104,1.0,0.0
246108,0.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246101,3,4
246102,0,25
246103,0,7
246104,1,0
246108,0,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246101,3,4
1,246102,0,25
2,246103,0,7
3,246104,1,0
4,246108,0,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246101,3,4
1,246102,,25
2,246103,,7
3,246104,1,
4,246108,,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc."
1,Sep 12 2022 8:21AM,246108,10,Beach Products Inc
14,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd"
21,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc."
22,Sep 12 2022 7:42AM,246102,19,"Innogenix, LLC"
47,Sep 12 2022 7:40AM,246101,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc.",,1
1,Sep 12 2022 8:21AM,246108,10,Beach Products Inc,,13
2,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd",,7
3,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc.",1,
4,Sep 12 2022 7:42AM,246102,19,"Innogenix, LLC",,25
5,Sep 12 2022 7:40AM,246101,20,Alumier Labs Inc.,3,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc.",1,
1,Sep 12 2022 8:21AM,246108,10,Beach Products Inc,13,
2,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd",7,
3,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc.",,1
4,Sep 12 2022 7:42AM,246102,19,"Innogenix, LLC",25,
5,Sep 12 2022 7:40AM,246101,20,Alumier Labs Inc.,4,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc.",1,
1,Sep 12 2022 8:21AM,246108,10,Beach Products Inc,13,
2,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd",7,
3,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc.",,1
4,Sep 12 2022 7:42AM,246102,19,"Innogenix, LLC",25,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc.",1.0,NaN
1,Sep 12 2022 8:21AM,246108,10,Beach Products Inc,13.0,NaN
2,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd",7.0,NaN
3,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc.",NaN,1.0
4,Sep 12 2022 7:42AM,246102,19,"Innogenix, LLC",25.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 12 2022 8:24AM,246110,10,"Amcyte Pharma, Inc.",1.0,0.0
1,Sep 12 2022 8:21AM,246108,10,Beach Products Inc,13.0,0.0
2,Sep 12 2022 7:50AM,246103,19,"AdvaGen Pharma, Ltd",7.0,0.0
3,Sep 12 2022 7:50AM,246104,22,"NBTY Global, Inc.",0.0,1.0
4,Sep 12 2022 7:42AM,246102,19,"Innogenix, LLC",25.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,492218,14.0,0.0
19,492205,32.0,0.0
20,246101,4.0,3.0
22,246104,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,492218,14.0,0.0
1,19,492205,32.0,0.0
2,20,246101,4.0,3.0
3,22,246104,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,14.0,0.0
1,19,32.0,0.0
2,20,4.0,3.0
3,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,14.0
1,19,Released,32.0
2,20,Released,4.0
3,22,Released,0.0
4,10,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19,20,22
Status,,,,
Executing,0.0,0.0,3.0,1.0
Released,14.0,32.0,4.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19,20,22
0,Executing,0.0,0.0,3.0,1.0
1,Released,14.0,32.0,4.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19,20,22
0,Executing,0.0,0.0,3.0,1.0
1,Released,14.0,32.0,4.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()